In [ ]:
import pandas as pd
import numpy as np

# 1. Prepare Data
file_path = "真社製首個樓宇維修公開資料庫.xlsx"
df = pd.read_excel(file_path)

# Clean bid amount and calculate log bid
df['bid_amount'] = pd.to_numeric(df['涉及費用'].astype(str).str.replace(r'[^\d.]', '', regex=True), errors='coerce')
df['log_bid'] = np.log(df['bid_amount'])
df = df.dropna(subset=['log_bid', '公司名稱', '大廈/屋苑名稱(入標年份)'])

# 2. Calculate Residuals (De-centering)
# Remove project-specific cost factors by subtracting the project mean
df['project_mean'] = df.groupby('大廈/屋苑名稱(入標年份)')['log_bid'].transform('mean')
df['residual'] = df['log_bid'] - df['project_mean']

# 3. Filter Configuration
MIN_JOINT_PROJECTS = 5      # Minimum joint bids required to calculate correlation
CORRELATION_THRESHOLD = 0.8 # High risk threshold

# 4. Build Pivot Matrix (Index=Project, Columns=Company, Values=Residual)
pivot_matrix = df.pivot_table(index='大廈/屋苑名稱(入標年份)', columns='公司名稱', values='residual')

# 5. Detection Algorithm
suspicious_pairs = []
companies = pivot_matrix.columns
n_companies = len(companies)

print(f"Scanning interactions for {n_companies} companies...")

# Iterate through unique pairs of companies
for i in range(n_companies):
    for j in range(i + 1, n_companies):
        firm_a = companies[i]
        firm_b = companies[j]
        
        # Extract data for the pair
        pair_data = pivot_matrix[[firm_a, firm_b]].dropna()
        joint_count = len(pair_data)
        
        if joint_count >= MIN_JOINT_PROJECTS:
            # Calculate correlation of residuals
            corr = pair_data[firm_a].corr(pair_data[firm_b])
            
            if corr > CORRELATION_THRESHOLD:
                suspicious_pairs.append({
                    'Firm_A': firm_a,
                    'Firm_B': firm_b,
                    'Joint_Projects': joint_count,
                    'Correlation': corr
                })

# 6. Output Results
suspicious_df = pd.DataFrame(suspicious_pairs)

if not suspicious_df.empty:
    suspicious_df = suspicious_df.sort_values(by='Correlation', ascending=False)
    print("\n[High Risk Syndicates Found]")
    print(suspicious_df.head(100).to_markdown(index=False))
    
    # Save to CSV
    suspicious_df.to_csv('suspicious_syndicates.csv', index=False)
    print("\nSaved to 'suspicious_syndicates.csv'")
else:
    print("No pairs found exceeding the correlation threshold.")

In [ ]:
import pandas as pd
import numpy as np
import re

# ==========================================
# 1. Data Loading & Cleaning
# ==========================================
file_path = "傳真社製首個樓宇維修公開資料庫.xlsx"
df = pd.read_excel(file_path)

# --- Cleaning A: Win Status ---
# Standardize: 1 = Winner, 0 = Lost
df['is_winner'] = pd.to_numeric(df['中標'], errors='coerce').fillna(0).astype(int)

# --- Cleaning B: Rank ---
# 1 = Lowest price, higher number = more expensive
df['rank_num'] = pd.to_numeric(df['排名(平至貴)'], errors='coerce')

# --- Cleaning C: Building Features ---
df['has_mall'] = df['其他設施'].astype(str).str.contains('商場|mall', case=False, na=False).astype(int)
df['has_club'] = df['其他設施'].astype(str).str.contains('會所|club', case=False, na=False).astype(int)
# Extract unit count
df['units_num'] = df['單位'].astype(str).str.extract(r'(\d+)').astype(float)

print(f"Data cleaning complete. Total records: {len(df)}")
print("-" * 30)

# ==========================================
# 2. Core Analysis Logic
# ==========================================

# Filter for winning bids only
winning_bids = df[df['is_winner'] == 1].copy()

# Aggregate company profile
company_profile = winning_bids.groupby('公司名稱').agg({
    '大廈/屋苑名稱(入標年份)': 'count',      # Win Count
    'rank_num': 'mean',                   # Avg Winning Rank
    'has_mall': 'mean',                   # Mall Ratio
    'has_club': 'mean',                   # Club Ratio
    'units_num': 'mean',                  # Avg Project Size (Units)
    '公司性質': 'first'                   # Company Type
}).rename(columns={'大廈/屋苑名稱(入標年份)': 'total_wins'})

# Calculate Global Benchmarks
global_avg_rank = winning_bids['rank_num'].mean()
global_avg_units = winning_bids['units_num'].mean()
print(f"Global Benchmarks: Avg Rank={global_avg_rank:.2f}, Avg Units={global_avg_units:.0f}")

# ==========================================
# 3. Define Risk Indicators
# ==========================================

# Filter: Companies with at least 2 wins
suspects = company_profile[company_profile['total_wins'] >= 2].copy()

# --- Indicator 1: Price Manipulation ---
# Avg winning rank > 3 implies winning despite high prices
suspects['Risk_HighPriceWin'] = suspects['rank_num'] > 3.0

# --- Indicator 2: Predatory Targeting ---
# Targeting large estates (>1000 units) with clubs (>30% ratio)
suspects['Risk_BigTarget'] = (suspects['units_num'] > 1000) & (suspects['has_club'] > 0.3)

# Combined High Risk Flag
suspects['High_Risk_Flag'] = suspects['Risk_HighPriceWin'] | suspects['Risk_BigTarget']

# ==========================================
# 4. Output Results
# ==========================================

top_suspects = suspects[suspects['High_Risk_Flag'] == True].sort_values(by='rank_num', ascending=False)

columns_to_show = ['total_wins', 'rank_num', 'units_num', 'has_club', 'Risk_HighPriceWin', 'Risk_BigTarget', '公司性質']
print("\n[High Risk Companies (Based on Feature Analysis)]")
print(top_suspects[columns_to_show])

# Save to CSV
top_suspects[columns_to_show].to_csv('high_risk_bidders_analysis.csv')
print("\nResults saved to 'high_risk_bidders_analysis.csv'")

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Data Preparation
file_path = "傳真社製首個樓宇維修公開資料庫.xlsx"
df = pd.read_excel(file_path)
df['bid_amount'] = pd.to_numeric(df['涉及費用'].astype(str).str.replace(r'[^\d.]', '', regex=True), errors='coerce')
df['log_bid'] = np.log(df['bid_amount'])
df['is_winner'] = pd.to_numeric(df['中標'], errors='coerce').fillna(0).astype(int)
df['rank_num'] = pd.to_numeric(df['排名(平至貴)'], errors='coerce')
df['units_num'] = df['單位'].astype(str).str.extract(r'(\d+)').astype(float)

# Extract district
def extract_district(name):
    districts = ['沙田', '荃灣', '土瓜灣', '深水埗', '灣仔', '西環', '大圍', '元朗', '青衣', '大埔', '九龍城', '上環', '中環']
    for d in districts:
        if d in str(name): return d
    return 'Other'
df['district'] = df['大廈/屋苑名稱(入標年份)'].apply(extract_district)

# 2. Build Network Graph Data (based on residual correlation)
# Calculate residuals
df_reg = df.dropna(subset=['log_bid', '公司名稱', '大廈/屋苑名稱(入標年份)']).copy()
df_reg['project_mean'] = df_reg.groupby('大廈/屋苑名稱(入標年份)')['log_bid'].transform('mean')
df_reg['residual'] = df_reg['log_bid'] - df_reg['project_mean']

# Pivot table
pivot_matrix = df_reg.pivot_table(index='大廈/屋苑名稱(入標年份)', columns='公司名稱', values='residual')

# Filter high-risk pairs (correlation > 0.8, common projects >= 3)
corr_matrix = pivot_matrix.corr()
links = []
companies = corr_matrix.columns
# For demonstration, take only active companies to avoid cluttered graph
active_companies = df['公司名稱'].value_counts().head(30).index.tolist()
# Ensure companies involved in the case are included
garden_vista_companies = df[df['涉翠湖案'] == 1]['公司名稱'].unique()
target_companies = list(set(active_companies) | set(garden_vista_companies))

filtered_corr = pivot_matrix[target_companies].corr()

# Build edge list
for i in range(len(target_companies)):
    for j in range(i+1, len(target_companies)):
        firm_a = target_companies[i]
        firm_b = target_companies[j]
        # Check number of common projects
        common_projects = pivot_matrix[[firm_a, firm_b]].dropna().shape[0]
        if common_projects >= 3:
            corr = filtered_corr.loc[firm_a, firm_b]
            if corr > 0.8: # Threshold
                links.append((firm_a, firm_b, corr))

# 3. Plot 1: Collusion Network Graph
plt.figure(figsize=(12, 12))
G = nx.Graph()
for firm_a, firm_b, corr in links:
    G.add_edge(firm_a, firm_b, weight=corr)

# Node colors: case-involved companies in red, others in blue
node_colors = []
for node in G.nodes():
    if node in garden_vista_companies:
        node_colors.append('#FF6B6B') # Red
    else:
        node_colors.append('#4ECDC4') # Teal

# Layout
pos = nx.spring_layout(G, k=0.3, iterations=50, seed=42)
nx.draw_networkx_nodes(G, pos, node_size=300, node_color=node_colors, alpha=0.9)
nx.draw_networkx_edges(G, pos, width=1.5, alpha=0.5, edge_color='gray')
# Label optimization
degrees = dict(G.degree)
labels = {node: node for node in G.nodes() if degrees[node] > 1}
# Set font for Chinese display
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
nx.draw_networkx_labels(G, pos, labels=labels, font_size=8)

plt.title("Collusion Network: High Correlation Pairs (>0.8)\n(Red nodes = Known Bad Companies)", fontsize=15)
plt.axis('off')
plt.savefig('collusion_network.png')

# 4. Plot 2: Scatter Plot (Winning Rank vs Project Size)
plt.figure(figsize=(10, 6))
winners = df[df['is_winner'] == 1].dropna(subset=['units_num', 'rank_num'])
# Mark case-involved companies
winners['Type'] = winners['公司名稱'].apply(lambda x: 'High Risk (Case Involved)' if x in garden_vista_companies else 'Normal')

sns.scatterplot(data=winners, x='units_num', y='rank_num', hue='Type', style='Type', 
                palette={'High Risk (Case Involved)': '#FF4500', 'Normal': '#1f77b4'}, s=100, alpha=0.7)

plt.axhline(y=3, color='gray', linestyle='--', alpha=0.5, label='Risk Threshold (Rank > 3)')
plt.axvline(x=1000, color='gray', linestyle='--', alpha=0.5, label='Target Threshold (Units > 1000)')
plt.title("Winning Strategy Analysis: Are they targeting 'Fat Sheep'?", fontsize=14)
plt.xlabel("Project Size (Number of Units)")
plt.ylabel("Winning Price Rank (1=Cheapest)")
plt.legend(title='Company Type')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('winning_strategy_scatter.png')

print("Charts generated: collusion_network.png, winning_strategy_scatter.png")

In [ ]:
import pandas as pd

def map_gvkey_to_cik():
    # 1. Read data
    print("Reading data...")
    try:
        # Read main dataset
        df_main = pd.read_csv('data_FraudDetection_JAR2020.csv')
        # Read auxiliary dataset containing CIK
        df_aaer = pd.read_csv('AAER_firm_year.csv')
    except FileNotFoundError as e:
        print(f"Error: File not found. Please ensure the CSV files are in the current directory.\nDetails: {e}")
        return

    # 2. Data preprocessing
    # Ensure the data types of the join keys are consistent (usually p_aaer is numeric)
    # In some cases, CSV reading may treat it as object, here force convert to numeric
    df_main['p_aaer'] = pd.to_numeric(df_main['p_aaer'], errors='coerce')
    df_aaer['P_AAER'] = pd.to_numeric(df_aaer['P_AAER'], errors='coerce')

    # 3. Prepare mapping table
    # The AAER file may have multiple rows for the same P_AAER (different years), but CIK should be the same.
    # We only take P_AAER and CIK columns, and drop duplicates to prevent inflation of main data rows.
    mapping_table = df_aaer[['P_AAER', 'CIK']].drop_duplicates()

    # 4. Perform merge (Merge/Join)
    # Use left join to retain all rows from main data
    print("Merging data...")
    df_result = pd.merge(
        df_main, 
        mapping_table, 
        left_on='p_aaer',   # Main table join key
        right_on='P_AAER',  # Mapping table join key
        how='left'          # Retain all rows from main data even if CIK is not found
    )

    # 5. Result statistics
    total_rows = len(df_result)
    matched_rows = df_result['CIK'].notna().sum()
    
    print("-" * 30)
    print(f"Processing completed!")
    print(f"Total rows: {total_rows}")
    print(f"Rows successfully matched to CIK (fraud samples): {matched_rows}")
    print(f"Rows not matched to CIK (non-fraud samples): {total_rows - matched_rows}")
    unique_cik_count = df_result['CIK'].dropna().nunique()
    print(f"Unique CIK count (deduplicated fraud samples): {unique_cik_count}")
    print("-" * 30)

    # 6. View sample data
    # Print a few rows of data successfully matched to CIK
    print("\nSample of successfully matched records (first 5 rows):")
    print(df_result[df_result['CIK'].notna()][['gvkey', 'p_aaer', 'CIK', 'misstate']].head())

    # 7. Save results (optional)
    df_result.to_csv('C:\\Users\\MaXin\\Desktop\\HSBC\\FraudDetection-master\\FraudDetection\\data_with_cik.csv', index=False)
    print("\nFile saved as data_with_cik.csv")

if __name__ == "__main__":
    map_gvkey_to_cik()

In [ ]:
    # Reorder columns to place 'CIK' right after 'gvkey'
    cols = list(df_result.columns)
    if 'gvkey' in cols and 'CIK' in cols:
        cols.remove('CIK')
        gvkey_index = cols.index('gvkey')
        cols.insert(gvkey_index + 1, 'CIK')
    df_result = df_result[cols]
    
    df_result.to_csv('data_with_cik.csv', index=False)
    print("\nFile saved as data_with_cik.csv")
    
    # Save fraud-only rows to a new CSV
    fraud_df = df_result[df_result['misstate'] == 1]
    fraud_df.to_csv('fraud_only_data.csv', index=False)
    print("Fraud-only data saved as fraud_only_data.csv")



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def analyze_ticker_matches(df):
    # 1. 筛选出成功匹配到 Ticker 的行
    # 注意：根据之前的逻辑，没匹配上的可能是 None 或 NaN
    mask_matched = df['Ticker'].notna()
    df_matched = df[mask_matched]
    
    # -------------------------------------------------------
    # 统计 1: 总体匹配情况
    # -------------------------------------------------------
    total_rows = len(df)
    matched_count = len(df_matched)
    unique_companies_matched = df_matched['CIK'].nunique()
    
    print("="*40)
    print("DATASET TICKER 匹配统计报告")
    print("="*40)
    print(f"总行数 (Total Rows): {total_rows}")
    print(f"成功匹配 Ticker 的行数: {matched_count}")
    print(f"匹配率: {matched_count / total_rows:.2%}")
    print(f"成功匹配的唯一公司数 (Unique Companies): {unique_companies_matched}")
    print("-" * 40)

    # -------------------------------------------------------
    # 统计 2: 造假样本 vs 非造假样本 的匹配情况
    # -------------------------------------------------------
    # 很多老旧的造假公司可能已经退市，导致匹配失败，这里看下差异
    if 'misstate' in df.columns:
        fraud_df = df[df['misstate'] == 1]
        fraud_matched = fraud_df[fraud_df['Ticker'].notna()]
        
        print(f"造假样本 (Fraud) 总数: {len(fraud_df)}")
        print(f"造假样本匹配到 Ticker 数: {len(fraud_matched)}")
        print(f"造假样本匹配率: {len(fraud_matched) / len(fraud_df) if len(fraud_df)>0 else 0:.2%}")
        print("-" * 40)

    # -------------------------------------------------------
    # 统计 3: 年份分布 (Year Distribution)
    # -------------------------------------------------------
    print("匹配成功的年份分布 (前10个年份):")
    year_dist = df_matched['fyear'].value_counts().sort_index()
    print(year_dist.head(10))
    print("...")
    print(year_dist.tail(5))
    
    # -------------------------------------------------------
    # 可视化 (可选)
    # -------------------------------------------------------
    # 绘制简单的柱状图看分布
    plt.figure(figsize=(12, 6))
    plt.bar(year_dist.index, year_dist.values, color='skyblue', label='Matched Rows')
    plt.title('Distribution of Rows with Matched Tickers by Fiscal Year')
    plt.xlabel('Fiscal Year (fyear)')
    plt.ylabel('Count of Matched Rows')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.legend()
    plt.show()

# 运行分析
analyze_ticker_matches(df_final)